<a href="https://colab.research.google.com/github/prashantmalan/Chatbot/blob/main/XmltoDf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
import xml.etree.ElementTree as ET
import pandas as pd
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Assuming the XML file is named 'data.xml'
tree = ET.parse('/content/drive/My Drive/Colab Notebooks/anomaly/ird-ex01-vanilla-swap.xml')
root = tree.getroot()

def get_data_for_xpaths(element, current_path="", data=None, counter=None):
  """
  Recursively extracts data for each XPath, handling duplicates with a global counter.

  Args:
    element: The current XML element.
    current_path: The XPath expression leading to the current element.
    data: A dictionary to store the extracted data.
    counter: A dictionary to keep track of duplicate paths.

  Returns:
    A dictionary where keys are unique XPaths and values are data.
  """
  if data is None:
    data = {}
  if counter is None:
    counter = {}
  if not current_path:
    current_path = element.tag

  attrib_str = ""
  if element.attrib:
    attrib_str = " ".join([f"@{k}='{v}'" for k, v in element.attrib.items()])
  path_with_attrib = f"{current_path} [{attrib_str}]"

  if path_with_attrib not in counter:
    counter[path_with_attrib] = 0
  else:
    counter[path_with_attrib] += 1
  unique_path = f"{path_with_attrib}{counter[path_with_attrib]}" if counter[path_with_attrib] > 0 else path_with_attrib

  data[unique_path] = element.text

  for child in element:
    child_path = f"{unique_path}/{child.tag}"
    get_data_for_xpaths(child, child_path, data, counter)  # Pass the same counter down
  return data

# Get data for XPaths
data = get_data_for_xpaths(root)

# Create DataFrame
df = pd.DataFrame(data, index=[0])
df

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,{http://www.fpml.org/FpML-5/transparency}publicExecutionReport [@fpmlVersion='5-2' @{http://www.w3.org/2001/XMLSchema-instance}schemaLocation='http://www.fpml.org/FpML-5/transparency ../../fpml-main-5-2.xsd'],{http://www.fpml.org/FpML-5/transparency}publicExecutionReport [@fpmlVersion='5-2' @{http://www.w3.org/2001/XMLSchema-instance}schemaLocation='http://www.fpml.org/FpML-5/transparency ../../fpml-main-5-2.xsd']/{http://www.fpml.org/FpML-5/transparency}header [],{http://www.fpml.org/FpML-5/transparency}publicExecutionReport [@fpmlVersion='5-2' @{http://www.w3.org/2001/XMLSchema-instance}schemaLocation='http://www.fpml.org/FpML-5/transparency ../../fpml-main-5-2.xsd']/{http://www.fpml.org/FpML-5/transparency}header []/{http://www.fpml.org/FpML-5/transparency}messageId [@messageIdScheme='http://abc.com/message-id'],{http://www.fpml.org/FpML-5/transparency}publicExecutionReport [@fpmlVersion='5-2' @{http://www.w3.org/2001/XMLSchema-instance}schemaLocation='http://www.fpml.org/FpML-5/transparency ../../fpml-main-5-2.xsd']/{http://www.fpml.org/FpML-5/transparency}header []/{http://www.fpml.org/FpML-5/transparency}sentBy [],{http://www.fpml.org/FpML-5/transparency}publicExecutionReport [@fpmlVersion='5-2' @{http://www.w3.org/2001/XMLSchema-instance}schemaLocation='http://www.fpml.org/FpML-5/transparency ../../fpml-main-5-2.xsd']/{http://www.fpml.org/FpML-5/transparency}header []/{http://www.fpml.org/FpML-5/transparency}sendTo [],{http://www.fpml.org/FpML-5/transparency}publicExecutionReport [@fpmlVersion='5-2' @{http://www.w3.org/2001/XMLSchema-instance}schemaLocation='http://www.fpml.org/FpML-5/transparency ../../fpml-main-5-2.xsd']/{http://www.fpml.org/FpML-5/transparency}header []/{http://www.fpml.org/FpML-5/transparency}creationTimestamp [],{http://www.fpml.org/FpML-5/transparency}publicExecutionReport [@fpmlVersion='5-2' @{http://www.w3.org/2001/XMLSchema-instance}schemaLocation='http://www.fpml.org/FpML-5/transparency ../../fpml-main-5-2.xsd']/{http://www.fpml.org/FpML-5/transparency}isCorrection [],{http://www.fpml.org/FpML-5/transparency}publicExecutionReport [@fpmlVersion='5-2' @{http://www.w3.org/2001/XMLSchema-instance}schemaLocation='http://www.fpml.org/FpML-5/transparency ../../fpml-main-5-2.xsd']/{http://www.fpml.org/FpML-5/transparency}correlationId [@correlationIdScheme='http://abc.com/correlation-id'],{http://www.fpml.org/FpML-5/transparency}publicExecutionReport [@fpmlVersion='5-2' @{http://www.w3.org/2001/XMLSchema-instance}schemaLocation='http://www.fpml.org/FpML-5/transparency ../../fpml-main-5-2.xsd']/{http://www.fpml.org/FpML-5/transparency}sequenceNumber [],{http://www.fpml.org/FpML-5/transparency}publicExecutionReport [@fpmlVersion='5-2' @{http://www.w3.org/2001/XMLSchema-instance}schemaLocation='http://www.fpml.org/FpML-5/transparency ../../fpml-main-5-2.xsd']/{http://www.fpml.org/FpML-5/transparency}originatingEvent [],...,{http://www.fpml.org/FpML-5/transparency}publicExecutionReport [@fpmlVersion='5-2' @{http://www.w3.org/2001/XMLSchema-instance}schemaLocation='http://www.fpml.org/FpML-5/transparency ../../fpml-main-5-2.xsd']/{http://www.fpml.org/FpML-5/transparency}trade []/{http://www.fpml.org/FpML-5/transparency}swap []/{http://www.fpml.org/FpML-5/transparency}swapStream []1/{http://www.fpml.org/FpML-5/transparency}paymentDates []/{http://www.fpml.org/FpML-5/transparency}paymentFrequency []/{http://www.fpml.org/FpML-5/transparency}period [],{http://www.fpml.org/FpML-5/transparency}publicExecutionReport [@fpmlVersion='5-2' @{http://www.w3.org/2001/XMLSchema-instance}schemaLocation='http://www.fpml.org/FpML-5/transparency ../../fpml-main-5-2.xsd']/{http://www.fpml.org/FpML-5/transparency}trade []/{http://www.fpml.org/FpML-5/transparency}swap []/{http://www.fpml.org/FpML-5/transparency}swapStream []1/{http://www.fpml.org/FpML-5/transparency}calculationPeriodAmount [],{http://www.fpml.org/FpML-5/transparency}publicExecutionReport [@fpmlVersion='5-2' @{http://www.w3.org/2001/XMLSchema-instance}schemaLoc

In [7]:
# prompt: I want to  df with 2 columns 1 is xpath  and 1 for field name  for example expected output should be {{DTCC_Wrapper}/BizData/Pyld/}Document/}DerivsTradRpt/}TradData/}Rpt/}New/}CtrPtySpcfcData/}CtrPty/}RptgCtrPty/}DrctnOrSd/}Drctn	Drctn
# file is WrapperSample_NEWT_COMM_MultipleEAs_Feb14

import xml.etree.ElementTree as ET
import pandas as pd
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Assuming the XML file is named 'data.xml'
tree = ET.parse('/content/drive/My Drive/Colab Notebooks/anomaly/WrapperSample_NEWT_COMM_MultipleEAs_Feb14.xml')
root = tree.getroot()

def extract_xpath_and_field_name(element, current_path="", data=None):
  """
  Recursively extracts XPath and field names from the XML tree.
  """
  if data is None:
    data = []

  if element.text:
    data.append([current_path, element.tag])

  for child in element:
    child_path = f"{current_path}/{child.tag}" if current_path else child.tag
    extract_xpath_and_field_name(child, child_path, data)

  return data

# Extract XPath and field name data
xpath_field_data = extract_xpath_and_field_name(root)

# Create DataFrame
df = pd.DataFrame(xpath_field_data, columns=['XPath', 'FieldName'])
print(df)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
                                                 XPath  \
0                                                        
1                    {DTCC_Wrapper}ExecutionAgentBlock   
2    {DTCC_Wrapper}ExecutionAgentBlock/{DTCC_Wrappe...   
3    {DTCC_Wrapper}ExecutionAgentBlock/{DTCC_Wrappe...   
4    {DTCC_Wrapper}ExecutionAgentBlock/{DTCC_Wrappe...   
..                                                 ...   
675  {urn:iso:std:iso:20022:tech:xsd:head.003.001.0...   
676  {urn:iso:std:iso:20022:tech:xsd:head.003.001.0...   
677  {urn:iso:std:iso:20022:tech:xsd:head.003.001.0...   
678  {urn:iso:std:iso:20022:tech:xsd:head.003.001.0...   
679  {urn:iso:std:iso:20022:tech:xsd:head.003.001.0...   

                                             FieldName  
0                           {DTCC_Wrapper}DTCC_Wrapper  
1                    {DTCC_Wrapper}ExecutionAgentBlock  
2  

In [13]:
# prompt: the first xpath should be {DTCC_Wrapper}DTCC_Wrapper  and field name DTCC_Wrapper  and dont hard code it

import xml.etree.ElementTree as ET
import pandas as pd
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Assuming the XML file is named 'data.xml'
tree = ET.parse('/content/drive/My Drive/Colab Notebooks/anomaly/WrapperSample_NEWT_COMM_MultipleEAs_Feb14.xml')
root = tree.getroot()

def extract_xpath_and_field_name(element, current_path="", data=None):
  """
  Recursively extracts XPath and field names from the XML tree.
  """
  if data is None:
    data = []

  if element.text:
    # Use the root element's tag as the initial XPath and field name
    if not current_path:
      current_path = element.tag
    data.append([current_path, element.tag])

  for child in element:
    child_path = f"{current_path}/{child.tag}" if current_path else child.tag
    extract_xpath_and_field_name(child, child_path, data)

  return data

# Extract XPath and field name data
xpath_field_data = extract_xpath_and_field_name(root)

# Create DataFrame
df = pd.DataFrame(xpath_field_data, columns=['XPath', 'FieldName'])
print(df)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,XPath,FieldName
0,{DTCC_Wrapper}DTCC_Wrapper,DTCC_Wrapper
1,{DTCC_Wrapper}ExecutionAgentBlock,{DTCC_Wrapper}ExecutionAgentBlock
2,{DTCC_Wrapper}ExecutionAgentBlock/{DTCC_Wrappe...,{DTCC_Wrapper}ExecutionAgent
3,{DTCC_Wrapper}ExecutionAgentBlock/{DTCC_Wrappe...,{DTCC_Wrapper}Uid
4,{DTCC_Wrapper}ExecutionAgentBlock/{DTCC_Wrappe...,{DTCC_Wrapper}Uid
...,...,...
675,{urn:iso:std:iso:20022:tech:xsd:head.003.001.0...,{urn:iso:std:iso:20022:tech:xsd:auth.030.001.0...
676,{urn:iso:std:iso:20022:tech:xsd:head.003.001.0...,{urn:iso:std:iso:20022:tech:xsd:auth.030.001.0...
677,{urn:iso:std:iso:20022:tech:xsd:head.003.001.0...,{urn:iso:std:iso:20022:tech:xsd:auth.030.001.0...
678,{urn:iso:std:iso:20022:tech:xsd:head.003.001.0...,{urn:iso:std:iso:20022:tech:xsd:auth.030.001.0...


In [17]:
# prompt: the {DTCC_Wrapper} is a root node , so it should be appended in child tag but this is not the case in line 8

import xml.etree.ElementTree as ET
import pandas as pd
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Assuming the XML file is named 'data.xml'
tree = ET.parse('/content/drive/My Drive/Colab Notebooks/anomaly/WrapperSample_NEWT_COMM_MultipleEAs_Feb14.xml')
root = tree.getroot()

def extract_xpath_and_field_name(element, current_path="", data=None):
  """
  Recursively extracts XPath and field names from the XML tree.
  """
  if data is None:
    data = []

  if element.text:
    # Use the root element's tag as the initial XPath and field name
    if not current_path:
      current_path = element.tag  # Use the root element's tag
    data.append([current_path, element.tag])

  for child in element:
    child_path = f"{current_path}/{child.tag}" if current_path else child.tag
    extract_xpath_and_field_name(child, child_path, data)

  return data

# Extract XPath and field name data
xpath_field_data = extract_xpath_and_field_name(root)

# Create DataFrame
df = pd.DataFrame(xpath_field_data, columns=['XPath', 'FieldName'])
df


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,XPath,FieldName
0,{DTCC_Wrapper}DTCC_Wrapper,{DTCC_Wrapper}DTCC_Wrapper
1,{DTCC_Wrapper}DTCC_Wrapper/{DTCC_Wrapper}Execu...,{DTCC_Wrapper}ExecutionAgentBlock
2,{DTCC_Wrapper}DTCC_Wrapper/{DTCC_Wrapper}Execu...,{DTCC_Wrapper}ExecutionAgent
3,{DTCC_Wrapper}DTCC_Wrapper/{DTCC_Wrapper}Execu...,{DTCC_Wrapper}Uid
4,{DTCC_Wrapper}DTCC_Wrapper/{DTCC_Wrapper}Execu...,{DTCC_Wrapper}Uid
...,...,...
675,{DTCC_Wrapper}DTCC_Wrapper/{urn:iso:std:iso:20...,{urn:iso:std:iso:20022:tech:xsd:auth.030.001.0...
676,{DTCC_Wrapper}DTCC_Wrapper/{urn:iso:std:iso:20...,{urn:iso:std:iso:20022:tech:xsd:auth.030.001.0...
677,{DTCC_Wrapper}DTCC_Wrapper/{urn:iso:std:iso:20...,{urn:iso:std:iso:20022:tech:xsd:auth.030.001.0...
678,{DTCC_Wrapper}DTCC_Wrapper/{urn:iso:std:iso:20...,{urn:iso:std:iso:20022:tech:xsd:auth.030.001.0...


In [18]:
# prompt: now root node is 	{DTCC_Wrapper} and field name is 	DTCC_Wrapper but u are giving root node as {DTCC_Wrapper}DTCC_Wrapper

import xml.etree.ElementTree as ET
import pandas as pd
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Assuming the XML file is named 'data.xml'
tree = ET.parse('/content/drive/My Drive/Colab Notebooks/anomaly/WrapperSample_NEWT_COMM_MultipleEAs_Feb14.xml')
root = tree.getroot()

def extract_xpath_and_field_name(element, current_path="", data=None):
  """
  Recursively extracts XPath and field names from the XML tree.
  """
  if data is None:
    data = []

  if element.text:
    # Use the root element's tag as the initial XPath and field name
    if not current_path:
      current_path = element.tag  # Use the root element's tag
    data.append([current_path, element.tag])

  for child in element:
    child_path = f"{current_path}/{child.tag}" if current_path else child.tag
    extract_xpath_and_field_name(child, child_path, data)

  return data

# Extract XPath and field name data
xpath_field_data = extract_xpath_and_field_name(root)

# Create DataFrame
df = pd.DataFrame(xpath_field_data, columns=['XPath', 'FieldName'])

# Modify the XPath column to remove the root element's tag if it's present at the beginning
df['XPath'] = df['XPath'].str.replace(f"{root.tag}/", "", regex=False)

df


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,XPath,FieldName
0,{DTCC_Wrapper}DTCC_Wrapper,{DTCC_Wrapper}DTCC_Wrapper
1,{DTCC_Wrapper}ExecutionAgentBlock,{DTCC_Wrapper}ExecutionAgentBlock
2,{DTCC_Wrapper}ExecutionAgentBlock/{DTCC_Wrappe...,{DTCC_Wrapper}ExecutionAgent
3,{DTCC_Wrapper}ExecutionAgentBlock/{DTCC_Wrappe...,{DTCC_Wrapper}Uid
4,{DTCC_Wrapper}ExecutionAgentBlock/{DTCC_Wrappe...,{DTCC_Wrapper}Uid
...,...,...
675,{urn:iso:std:iso:20022:tech:xsd:head.003.001.0...,{urn:iso:std:iso:20022:tech:xsd:auth.030.001.0...
676,{urn:iso:std:iso:20022:tech:xsd:head.003.001.0...,{urn:iso:std:iso:20022:tech:xsd:auth.030.001.0...
677,{urn:iso:std:iso:20022:tech:xsd:head.003.001.0...,{urn:iso:std:iso:20022:tech:xsd:auth.030.001.0...
678,{urn:iso:std:iso:20022:tech:xsd:head.003.001.0...,{urn:iso:std:iso:20022:tech:xsd:auth.030.001.0...


In [21]:
# prompt: the expectded values are  XPath	Field Name
# {DTCC_Wrapper}DTCC_Wrapper	DTCC_Wrapper
# {{DTCC_Wrapper}/{DTCC_Wrapper}ExecutionAgentBlock	ExecutionAgentBlock
# {{DTCC_Wrapper}/{DTCC_Wrapper}ExecutionAgentBlock/{DTCC_Wrapper}ExecutionAgent	ExecutionAgent
# {{DTCC_Wrapper}/{DTCC_Wrapper}ExecutionAgentBlock/{DTCC_Wrapper}ExecutionAgent/{DTCC_Wrapper}Uid	Uid
# {{DTCC_Wrapper}/{DTCC_Wrapper}ExecutionAgentBlock/{D

import xml.etree.ElementTree as ET
import pandas as pd
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

expected_results= pd.read_csv('/content/drive/My Drive/Colab Notebooks/anomaly/expected_values_xpath.csv')
# Assuming the XML file is named 'data.xml'
tree = ET.parse('/content/drive/My Drive/Colab Notebooks/anomaly/WrapperSample_NEWT_COMM_MultipleEAs_Feb14.xml')
root = tree.getroot()

def extract_xpath_and_field_name(element, current_path="", data=None):
  """
  Recursively extracts XPath and field names from the XML tree.
  """
  if data is None:
    data = []

  if element.text:
    # Use the root element's tag as the initial XPath and field name
    if not current_path:
      current_path = element.tag  # Use the root element's tag
    data.append([f"{{{current_path}}}", element.tag])

  for child in element:
    child_path = f"{current_path}/{child.tag}" if current_path else child.tag
    extract_xpath_and_field_name(child, child_path, data)

  return data

# Extract XPath and field name data
xpath_field_data = extract_xpath_and_field_name(root)

# Create DataFrame
df = pd.DataFrame(xpath_field_data, columns=['XPath', 'FieldName'])

# Modify the XPath column to remove the root element's tag if it's present at the beginning
# df['XPath'] = df['XPath'].str.replace(f"{root.tag}/", "", regex=False)

df


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,XPath,FieldName
0,{{DTCC_Wrapper}DTCC_Wrapper},{DTCC_Wrapper}DTCC_Wrapper
1,{{DTCC_Wrapper}DTCC_Wrapper/{DTCC_Wrapper}Exec...,{DTCC_Wrapper}ExecutionAgentBlock
2,{{DTCC_Wrapper}DTCC_Wrapper/{DTCC_Wrapper}Exec...,{DTCC_Wrapper}ExecutionAgent
3,{{DTCC_Wrapper}DTCC_Wrapper/{DTCC_Wrapper}Exec...,{DTCC_Wrapper}Uid
4,{{DTCC_Wrapper}DTCC_Wrapper/{DTCC_Wrapper}Exec...,{DTCC_Wrapper}Uid
...,...,...
675,{{DTCC_Wrapper}DTCC_Wrapper/{urn:iso:std:iso:2...,{urn:iso:std:iso:20022:tech:xsd:auth.030.001.0...
676,{{DTCC_Wrapper}DTCC_Wrapper/{urn:iso:std:iso:2...,{urn:iso:std:iso:20022:tech:xsd:auth.030.001.0...
677,{{DTCC_Wrapper}DTCC_Wrapper/{urn:iso:std:iso:2...,{urn:iso:std:iso:20022:tech:xsd:auth.030.001.0...
678,{{DTCC_Wrapper}DTCC_Wrapper/{urn:iso:std:iso:2...,{urn:iso:std:iso:20022:tech:xsd:auth.030.001.0...


In [24]:
# prompt: match the results with expected values and write code to input the xml and output expected values

import xml.etree.ElementTree as ET
import pandas as pd
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

def extract_xpath_and_field_name(element, current_path="", data=None):
  """
  Recursively extracts XPath and field names from the XML tree.
  """
  if data is None:
    data = []

  if element.text:
    # Use the root element's tag as the initial XPath and field name
    if not current_path:
      current_path = element.tag  # Use the root element's tag
    data.append([f"{{{current_path}}}", element.tag])

  for child in element:
    child_path = f"{current_path}/{child.tag}" if current_path else child.tag
    extract_xpath_and_field_name(child, child_path, data)

  return data

def process_xml_file(xml_file_path):
  """
  Processes an XML file, extracts XPath and field names, and matches with expected values.
  """
  tree = ET.parse(xml_file_path)
  root = tree.getroot()
  xpath_field_data = extract_xpath_and_field_name(root)
  df = pd.DataFrame(xpath_field_data, columns=['XPath', 'FieldName'])
  return df

def match_with_expected_values(df, expected_results_path):
  """
  Matches the extracted XPath and field names with expected values from a CSV file.
  """
  expected_results = pd.read_csv(expected_results_path)
  merged_df = pd.merge(df, expected_results, on='XPath', how='inner')
  return merged_df

# Example usage:
xml_file_path = '/content/drive/My Drive/Colab Notebooks/anomaly/WrapperSample_NEWT_COMM_MultipleEAs_Feb14.xml'
expected_results_path = '/content/drive/My Drive/Colab Notebooks/anomaly/expected_values_xpath.csv'

df = process_xml_file(xml_file_path)
merged_df = match_with_expected_values(df, expected_results_path)

merged_df
cv


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


NameError: name 'cv' is not defined

In [25]:
# prompt: match the results with expected values and write code to input the xml and output expected values

import xml.etree.ElementTree as ET


In [27]:
# prompt: match the results with expected values and write code to input the xml and output expected values

import xml.etree.ElementTree as ET
import pandas as pd
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

def extract_xpath_and_field_name(element, current_path="", data=None):
  """
  Recursively extracts XPath and field names from the XML tree.
  """
  if data is None:
    data = []

  if element.text:
    # Use the root element's tag as the initial XPath and field name
    if not current_path:
      current_path = element.tag  # Use the root element's tag
    data.append([f"{{{current_path}}}", element.tag])

  for child in element:
    child_path = f"{current_path}/{child.tag}" if current_path else child.tag
    extract_xpath_and_field_name(child, child_path, data)

  return data

def process_xml_file(xml_file_path):
  """
  Processes an XML file, extracts XPath and field names, and matches with expected values.
  """
  tree = ET.parse(xml_file_path)
  root = tree.getroot()
  xpath_field_data = extract_xpath_and_field_name(root)
  df = pd.DataFrame(xpath_field_data, columns=['XPath', 'FieldName'])
  return df

def match_with_expected_values(df, expected_results_path):
  """
  Matches the extracted XPath and field names with expected values from a CSV file.
  """
  expected_results = pd.read_csv(expected_results_path)
  merged_df = pd.merge(df, expected_results, on='XPath', how='inner')
  return merged_df

# Example usage:
xml_file_path = '/content/drive/My Drive/Colab Notebooks/anomaly/WrapperSample_NEWT_COMM_MultipleEAs_Feb14.xml'
expected_results_path = '/content/drive/My Drive/Colab Notebooks/anomaly/expected_values_xpath.csv'

df = process_xml_file(xml_file_path)
merged_df = match_with_expected_values(df, expected_results_path)

merged_df # Display the merged_df dataframe

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,XPath,FieldName,Field Name


In [28]:
# prompt: match the results with expected values and write code to input the xml and output expected values

import xml.etree.ElementTree as ET
import pandas as pd
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

def extract_xpath_and_field_name(element, current_path="", data=None):
  """
  Recursively extracts XPath and field names from the XML tree.
  """
  if data is None:
    data = []

  if element.text:
    # Use the root element's tag as the initial XPath and field name
    if not current_path:
      current_path = element.tag  # Use the root element's tag
    data.append([f"{{{current_path}}}", element.tag])

  for child in element:
    child_path = f"{current_path}/{child.tag}" if current_path else child.tag
    extract_xpath_and_field_name(child, child_path, data)

  return data

def process_xml_file(xml_file_path):
  """
  Processes an XML file, extracts XPath and field names, and matches with expected values.
  """
  tree = ET.parse(xml_file_path)
  root = tree.getroot()
  xpath_field_data = extract_xpath_and_field_name(root)
  df = pd.DataFrame(xpath_field_data, columns=['XPath', 'FieldName'])
  return df

def match_with_expected_values(df, expected_results_path):
  """
  Matches the extracted XPath and field names with expected values from a CSV file.
  """
  expected_results = pd.read_csv(expected_results_path)

  # Print the 'XPath' column of both dataframes
  print(f"df['XPath']:\n{df['XPath']}")
  print(f"\nexpected_results['XPath']:\n{expected_results['XPath']}")

  merged_df = pd.merge(df, expected_results, on='XPath', how='inner')
  return merged_df

# Example usage:
xml_file_path = '/content/drive/My Drive/Colab Notebooks/anomaly/WrapperSample_NEWT_COMM_MultipleEAs_Feb14.xml'
expected_results_path = '/content/drive/My Drive/Colab Notebooks/anomaly/expected_values_xpath.csv'

df = process_xml_file(xml_file_path)
merged_df = match_with_expected_values(df, expected_results_path)

merged_df

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
df['XPath']:
0                           {{DTCC_Wrapper}DTCC_Wrapper}
1      {{DTCC_Wrapper}DTCC_Wrapper/{DTCC_Wrapper}Exec...
2      {{DTCC_Wrapper}DTCC_Wrapper/{DTCC_Wrapper}Exec...
3      {{DTCC_Wrapper}DTCC_Wrapper/{DTCC_Wrapper}Exec...
4      {{DTCC_Wrapper}DTCC_Wrapper/{DTCC_Wrapper}Exec...
                             ...                        
675    {{DTCC_Wrapper}DTCC_Wrapper/{urn:iso:std:iso:2...
676    {{DTCC_Wrapper}DTCC_Wrapper/{urn:iso:std:iso:2...
677    {{DTCC_Wrapper}DTCC_Wrapper/{urn:iso:std:iso:2...
678    {{DTCC_Wrapper}DTCC_Wrapper/{urn:iso:std:iso:2...
679    {{DTCC_Wrapper}DTCC_Wrapper/{urn:iso:std:iso:2...
Name: XPath, Length: 680, dtype: object

expected_results['XPath']:
0                               DTCC_WrapperDTCC_Wrapper
1           DTCC_Wrapper/DTCC_WrapperExecutionAgentBlock
2      DTCC_Wrapper/DTCC_WrapperExecutionAgentBlo

,XPath,FieldName,Field Name


In [29]:
import xml.etree.ElementTree as ET
import pandas as pd
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

def extract_xpath_and_field_name(element, current_path="", data=None):
  """
  Recursively extracts XPath and field names from the XML tree.
  """
  if data is None:
    data = []

  if element.text:
    # Use the root element's tag as the initial XPath and field name
    if not current_path:
      current_path = element.tag  # Use the root element's tag
    data.append([f"{{{current_path}}}", element.tag]) # Add curly braces around the entire XPath

  for child in element:
    child_path = f"{current_path}/{child.tag}" if current_path else child.tag
    extract_xpath_and_field_name(child, child_path, data)

  return data

def process_xml_file(xml_file_path):
  """
  Processes an XML file, extracts XPath and field names, and matches with expected values.
  """
  tree = ET.parse(xml_file_path)
  root = tree.getroot()
  xpath_field_data = extract_xpath_and_field_name(root)
  df = pd.DataFrame(xpath_field_data, columns=['XPath', 'FieldName'])
  return df

def match_with_expected_values(df, expected_results_path):
  """
  Matches the extracted XPath and field names with expected values from a CSV file.
  """
  expected_results = pd.read_csv(expected_results_path)
  merged_df = pd.merge(df, expected_results, on='XPath', how='inner')
  return merged_df

# Example usage:
xml_file_path = '/content/drive/My Drive/Colab Notebooks/anomaly/WrapperSample_NEWT_COMM_MultipleEAs_Feb14.xml'
expected_results_path = '/content/drive/My Drive/Colab Notebooks/anomaly/expected_values_xpath.csv'

df = process_xml_file(xml_file_path)
merged_df = match_with_expected_values(df, expected_results_path)

merged_df

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,XPath,FieldName,Field Name


In [30]:
df

,XPath,FieldName
0,{{DTCC_Wrapper}DTCC_Wrapper},{DTCC_Wrapper}DTCC_Wrapper
1,{{DTCC_Wrapper}DTCC_Wrapper/{DTCC_Wrapper}Exec...,{DTCC_Wrapper}ExecutionAgentBlock
2,{{DTCC_Wrapper}DTCC_Wrapper/{DTCC_Wrapper}Exec...,{DTCC_Wrapper}ExecutionAgent
3,{{DTCC_Wrapper}DTCC_Wrapper/{DTCC_Wrapper}Exec...,{DTCC_Wrapper}Uid
4,{{DTCC_Wrapper}DTCC_Wrapper/{DTCC_Wrapper}Exec...,{DTCC_Wrapper}Uid
...,...,...
675,{{DTCC_Wrapper}DTCC_Wrapper/{urn:iso:std:iso:2...,{urn:iso:std:iso:20022:tech:xsd:auth.030.001.0...
676,{{DTCC_Wrapper}DTCC_Wrapper/{urn:iso:std:iso:2...,{urn:iso:std:iso:20022:tech:xsd:auth.030.001.0...
677,{{DTCC_Wrapper}DTCC_Wrapper/{urn:iso:std:iso:2...,{urn:iso:std:iso:20022:tech:xsd:auth.030.001.0...
678,{{DTCC_Wrapper}DTCC_Wrapper/{urn:iso:std:iso:2...,{urn:iso:std:iso:20022:tech:xsd:auth.030.001.0...


In [32]:
import xml.etree.ElementTree as ET
import pandas as pd
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

def extract_xpath_and_field_name(element, current_path="", data=None):
  """
  Recursively extracts XPath and field names from the XML tree.
  """
  if data is None:
    data = []

  if element.text:
    # Use the root element's tag as the initial XPath and field name
    if not current_path:
      current_path = element.tag  # Use the root element's tag
    data.append([f"{{{{{current_path}}}}}", element.tag]) # Add extra curly braces at the beginning

  for child in element:
    child_path = f"{current_path}/{child.tag}" if current_path else child.tag
    extract_xpath_and_field_name(child, child_path, data)

  return data

def process_xml_file(xml_file_path):
  """
  Processes an XML file, extracts XPath and field names, and matches with expected values.
  """
  tree = ET.parse(xml_file_path)
  root = tree.getroot()
  xpath_field_data = extract_xpath_and_field_name(root)
  df = pd.DataFrame(xpath_field_data, columns=['XPath', 'FieldName'])
  return df

def match_with_expected_values(df, expected_results_path):
  """
  Matches the extracted XPath and field names with expected values from a CSV file.
  """
  expected_results = pd.read_csv(expected_results_path)
  merged_df = pd.merge(df, expected_results, on='XPath', how='inner')
  return merged_df

# Example usage:
xml_file_path = '/content/drive/My Drive/Colab Notebooks/anomaly/WrapperSample_NEWT_COMM_MultipleEAs_Feb14.xml'
expected_results_path = '/content/drive/My Drive/Colab Notebooks/anomaly/expected_values_xpath.csv'

df = process_xml_file(xml_file_path)
merged_df = match_with_expected_values(df, expected_results_path)

df

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,XPath,FieldName
0,{{{DTCC_Wrapper}DTCC_Wrapper}},{DTCC_Wrapper}DTCC_Wrapper
1,{{{DTCC_Wrapper}DTCC_Wrapper/{DTCC_Wrapper}Exe...,{DTCC_Wrapper}ExecutionAgentBlock
2,{{{DTCC_Wrapper}DTCC_Wrapper/{DTCC_Wrapper}Exe...,{DTCC_Wrapper}ExecutionAgent
3,{{{DTCC_Wrapper}DTCC_Wrapper/{DTCC_Wrapper}Exe...,{DTCC_Wrapper}Uid
4,{{{DTCC_Wrapper}DTCC_Wrapper/{DTCC_Wrapper}Exe...,{DTCC_Wrapper}Uid
...,...,...
675,{{{DTCC_Wrapper}DTCC_Wrapper/{urn:iso:std:iso:...,{urn:iso:std:iso:20022:tech:xsd:auth.030.001.0...
676,{{{DTCC_Wrapper}DTCC_Wrapper/{urn:iso:std:iso:...,{urn:iso:std:iso:20022:tech:xsd:auth.030.001.0...
677,{{{DTCC_Wrapper}DTCC_Wrapper/{urn:iso:std:iso:...,{urn:iso:std:iso:20022:tech:xsd:auth.030.001.0...
678,{{{DTCC_Wrapper}DTCC_Wrapper/{urn:iso:std:iso:...,{urn:iso:std:iso:20022:tech:xsd:auth.030.001.0...


In [38]:
import xml.etree.ElementTree as ET
import pandas as pd
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

def extract_xpath_and_field_name(element, current_path="", data=None):
  """
  Recursively extracts XPath and field names from the XML tree.
  """
  if data is None:
    data = []

  if element.text:
    if not current_path:
      # Handle root node
      current_path = element.tag
      data.append([f"{{{element.tag}}}{element.tag}", element.tag])
    else:
      data.append([f"{{{current_path}}}", element.tag])

  for child in element:
    child_path = f"{current_path}/{child.tag}" if current_path else child.tag
    extract_xpath_and_field_name(child, child_path, data)

  return data

def process_xml_file(xml_file_path):
  """
  Processes an XML file, extracts XPath and field names, and matches with expected values.
  """
  tree = ET.parse(xml_file_path)
  root = tree.getroot()
  xpath_field_data = extract_xpath_and_field_name(root)
  df = pd.DataFrame(xpath_field_data, columns=['XPath', 'FieldName'])
  return df

def match_with_expected_values(df, expected_results_path):
  """
  Matches the extracted XPath and field names with expected values from a CSV file.
  """
  expected_results = pd.read_csv(expected_results_path)
  merged_df = pd.merge(df, expected_results, on='XPath', how='inner')
df

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,XPath,FieldName
0,{{DTCC_Wrapper}DTCC_Wrapper}{DTCC_Wrapper}DTCC...,{DTCC_Wrapper}DTCC_Wrapper
1,{{{DTCC_Wrapper}DTCC_Wrapper/{DTCC_Wrapper}Exe...,{DTCC_Wrapper}ExecutionAgentBlock
2,{{{DTCC_Wrapper}DTCC_Wrapper/{DTCC_Wrapper}Exe...,{DTCC_Wrapper}ExecutionAgent
3,{{{DTCC_Wrapper}DTCC_Wrapper/{DTCC_Wrapper}Exe...,{DTCC_Wrapper}Uid
4,{{{DTCC_Wrapper}DTCC_Wrapper/{DTCC_Wrapper}Exe...,{DTCC_Wrapper}Uid
...,...,...
675,{{{DTCC_Wrapper}DTCC_Wrapper/{urn:iso:std:iso:...,{urn:iso:std:iso:20022:tech:xsd:auth.030.001.0...
676,{{{DTCC_Wrapper}DTCC_Wrapper/{urn:iso:std:iso:...,{urn:iso:std:iso:20022:tech:xsd:auth.030.001.0...
677,{{{DTCC_Wrapper}DTCC_Wrapper/{urn:iso:std:iso:...,{urn:iso:std:iso:20022:tech:xsd:auth.030.001.0...
678,{{{DTCC_Wrapper}DTCC_Wrapper/{urn:iso:std:iso:...,{urn:iso:std:iso:20022:tech:xsd:auth.030.001.0...


In [2]:
import xml.etree.ElementTree as ET
import pandas as pd
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

def extract_xpath_and_field_name(element, current_path="", data=None):
  """
  Recursively extracts XPath and field names from the XML tree.
  """
  if data is None:
    data = []

  if element.text:
    if not current_path:
      # Handle root node
      data.append([f"{{{element.tag}}}{element.tag}", element.tag])
    else:
      data.append([f"{{{current_path}}}", element.tag])

  for child in element:
    child_path = f"{current_path}/{child.tag}" if current_path else child.tag
    extract_xpath_and_field_name(child, child_path, data)

  return data

def process_xml_file(xml_file_path):
  """
  Processes an XML file, extracts XPath and field names, and matches with expected values.
  """
  tree = ET.parse(xml_file_path)
  root = tree.getroot()
  xpath_field_data = extract_xpath_and_field_name(root)
  df = pd.DataFrame(xpath_field_data, columns=['XPath', 'FieldName'])
  return df

def match_with_expected_values(df, expected_results_path):
  """
  Matches the extracted XPath and field names with expected values from a CSV file.
  """
  expected_results = pd.read_csv(expected_results_path)
  merged_df = pd.merge(df, expected_results, on='XPath', how='inner')
  return merged_df

# Example usage:
xml_file_path = '/content/drive/My Drive/Colab Notebooks/anomaly/WrapperSample_NEWT_COMM_MultipleEAs_Feb14.xml'
expected_results_path = '/content/drive/My Drive/Colab Notebooks/anomaly/expected_values_xpath.csv'

df = process_xml_file(xml_file_path)
merged_df = match_with_expected_values(df, expected_results_path)

df

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,XPath,FieldName
0,{{DTCC_Wrapper}DTCC_Wrapper}{DTCC_Wrapper}DTCC...,{DTCC_Wrapper}DTCC_Wrapper
1,{{DTCC_Wrapper}ExecutionAgentBlock},{DTCC_Wrapper}ExecutionAgentBlock
2,{{DTCC_Wrapper}ExecutionAgentBlock/{DTCC_Wrapp...,{DTCC_Wrapper}ExecutionAgent
3,{{DTCC_Wrapper}ExecutionAgentBlock/{DTCC_Wrapp...,{DTCC_Wrapper}Uid
4,{{DTCC_Wrapper}ExecutionAgentBlock/{DTCC_Wrapp...,{DTCC_Wrapper}Uid
...,...,...
675,{{urn:iso:std:iso:20022:tech:xsd:head.003.001....,{urn:iso:std:iso:20022:tech:xsd:auth.030.001.0...
676,{{urn:iso:std:iso:20022:tech:xsd:head.003.001....,{urn:iso:std:iso:20022:tech:xsd:auth.030.001.0...
677,{{urn:iso:std:iso:20022:tech:xsd:head.003.001....,{urn:iso:std:iso:20022:tech:xsd:auth.030.001.0...
678,{{urn:iso:std:iso:20022:tech:xsd:head.003.001....,{urn:iso:std:iso:20022:tech:xsd:auth.030.001.0...


In [5]:
## take it

import xml.etree.ElementTree as ET
import pandas as pd
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

def extract_xpath_and_field_name(element, current_path="", data=None):
  """
  Recursively extracts XPath and field names from the XML tree.
  """
  if data is None:
    data = []

  if element.text:
    if not current_path:
      # Handle root node
      data.append([f"{{{element.tag}}}", element.tag]) # Corrected line
    else:
      data.append([f"{{{current_path}}}", element.tag])

  for child in element:
    child_path = f"{current_path}/{child.tag}" if current_path else child.tag
    extract_xpath_and_field_name(child, child_path, data)

  return data

def process_xml_file(xml_file_path):
  """
  Processes an XML file, extracts XPath and field names, and matches with expected values.
  """
  tree = ET.parse(xml_file_path)
  root = tree.getroot()
  xpath_field_data = extract_xpath_and_field_name(root)
  df = pd.DataFrame(xpath_field_data, columns=['XPath', 'FieldName'])
  return df

def match_with_expected_values(df, expected_results_path):
  """
  Matches the extracted XPath and field names with expected values from a CSV file.
  """
  expected_results = pd.read_csv(expected_results_path)
  merged_df = pd.merge(df, expected_results, on='XPath', how='inner')
  return merged_df

# Example usage:
xml_file_path = '/content/drive/My Drive/Colab Notebooks/anomaly/WrapperSample_NEWT_COMM_MultipleEAs_Feb14.xml'
expected_results_path = '/content/drive/My Drive/Colab Notebooks/anomaly/expected_values_xpath.csv'

df = process_xml_file(xml_file_path)
merged_df = match_with_expected_values(df, expected_results_path)

df

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,XPath,FieldName
0,{{DTCC_Wrapper}DTCC_Wrapper},{DTCC_Wrapper}DTCC_Wrapper
1,{{DTCC_Wrapper}ExecutionAgentBlock},{DTCC_Wrapper}ExecutionAgentBlock
2,{{DTCC_Wrapper}ExecutionAgentBlock/{DTCC_Wrapp...,{DTCC_Wrapper}ExecutionAgent
3,{{DTCC_Wrapper}ExecutionAgentBlock/{DTCC_Wrapp...,{DTCC_Wrapper}Uid
4,{{DTCC_Wrapper}ExecutionAgentBlock/{DTCC_Wrapp...,{DTCC_Wrapper}Uid
...,...,...
675,{{urn:iso:std:iso:20022:tech:xsd:head.003.001....,{urn:iso:std:iso:20022:tech:xsd:auth.030.001.0...
676,{{urn:iso:std:iso:20022:tech:xsd:head.003.001....,{urn:iso:std:iso:20022:tech:xsd:auth.030.001.0...
677,{{urn:iso:std:iso:20022:tech:xsd:head.003.001....,{urn:iso:std:iso:20022:tech:xsd:auth.030.001.0...
678,{{urn:iso:std:iso:20022:tech:xsd:head.003.001....,{urn:iso:std:iso:20022:tech:xsd:auth.030.001.0...
